<a href="https://colab.research.google.com/github/ananya0-0/Hand-Gesture-Recognition-/blob/main/Hand_Gesture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GITHUB REPO SETUP


In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ananya365","key":"30292cd59f8111026a3649b4006c0b95"}'}

In [2]:
!git config --global user.email "ananychauhan365@gmail.com"
!git config --global user.name "ananya365"
#cloning github repository
!rm -rf Hand-Gesture-Recognition- #deleting incorrect folder
!git clone https://github.com/ananya0-0/Hand-Gesture-Recognition.git

%cd Hand-Gesture-Recognition

Cloning into 'Hand-Gesture-Recognition'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'Hand-Gesture-Recognition'
/content


In [10]:
!pip install opencv-python mediapipe numpy

UPLOADING KAGGLE API TOKEN

In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#installing dataset from kaggle
!kaggle datasets download -d grassknoted/asl-alphabet

!unzip asl-alphabet.zip -d asl_alphabet_train

In [5]:
%cd /content/Hand-Gesture-Recognition

[Errno 2] No such file or directory: '/content/Hand-Gesture-Recognition'
/content


In [6]:
%%writefile .gitignore
__pycache__/
*.pkl
models/
*.zip
*.csv
/content/asl_data/

Writing .gitignore


In [7]:
print("\nCommitting .gitignore file...")
!git add .gitignore
!git commit -m "chore: Add .gitignore to exclude models and data"
print("Commit 1 complete.")


Committing .gitignore file...
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
Commit 1 complete.


REQUIRED ML PIPELINES SETUP


In [13]:
%%writefile train_model.py
import cv2
import numpy as np
import os
import mediapipe as mp
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Configuration part
dataset_dir = '/content/asl_alphabet_train/asl_alphabet_train'

model_save_dir = '/content/ASL_Model'
model_path = os.path.join(model_save_dir, 'asl_classifier.pkl')
label_map_path = os.path.join(model_save_dir , 'label_map.pkl')

#feature extraction
def extract_features(img_path,hands_model):
  img = cv2.imread(img_path)
  if img is None:
    return None

  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  results = hands_model.process(img_rgb)

  if results.multi_hand_landmarks:
    hand_landmarks = results.multi_hand_landmarks[0]
    landmark_vector = []

    wrist_x = hand_landmarks.landmark[0].x
    wrist_y = hand_landmarks.landmark[0].y

    for landmark in hand_landmarks.landmark:
      landmark_vector.extend([landmark.x - wrist_x, landmark.y - wrist_y])
    return np.array(landmark_vector)

Overwriting train_model.py


DATA PROCESSING AND TRAINING

In [19]:
import mediapipe as mp
import pickle
import os
dataset_dir = '/content/asl_alphabet_train/asl_alphabet_train'

model_save_dir = '/content/ASL_Model'
model_path = os.path.join(model_save_dir, 'asl_classifier.pkl')
label_map_path = os.path.join(model_save_dir , 'label_map.pkl')

# Feature extraction function
def extract_features(img_path,hands_model):
  img = cv2.imread(img_path)
  if img is None:
    return None

  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  results = hands_model.process(img_rgb)

  if results.multi_hand_landmarks:
    hand_landmarks = results.multi_hand_landmarks[0]
    landmark_vector = []

    wrist_x = hand_landmarks.landmark[0].x
    wrist_y = hand_landmarks.landmark[0].y

    for landmark in hand_landmarks.landmark:
      landmark_vector.extend([landmark.x - wrist_x, landmark.y - wrist_y])
    return np.array(landmark_vector)

  return None

def train_model():
  features = []
  labels = []
  label_map = {} # will store the names
  current_label_id = 0

  # Initializing hands model
  mp_hands = mp.solutions.hands
  hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)
  print(f"Loading data from: {dataset_dir}")

  if not os.path.exists(dataset_dir):
    print(f"Error: Dataset directory not found at {dataset_dir}")
    hands.close()
    return

  #label_map for all classes
  all_class_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
  for class_name in sorted(all_class_dirs):
    if class_name not in label_map:
      label_map[class_name] = current_label_id
      current_label_id += 1

  #iterate through classes and extract features
  for class_name in sorted(all_class_dirs):
    class_path = os.path.join(dataset_dir, class_name)
    label_id = label_map[class_name]
    print(f"Processing class: {class_name} (ID: {label_id})")

    for image_file in os.listdir(class_path):
      if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(class_path, image_file)
        feature_vector = extract_features(image_path, Hands)
        if feature_vector is not None:
          features.append(feature_vector)
          labels.append(label_id)

  hands.close()

  print(f"\nFeature extraction complete. Total samples: {len(features)}")
  if len(features) == 0:
    print("No features extracted. Check dataset path and image content.")
    return

  # Model Training
  X = np.array(features)
  y = np.array(labels)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
  mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42, verbose=True)
  mlp.fit(X_train, y_train)

  y_pred = mlp.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  print(f"Model Test Accuracy: {accuracy * 100:.2f}%")

  os.makedirs(model_save_dir, exist_ok=True)

  with open(model_path, 'wb') as file:
      pickle.dump(mlp, file)
  print(f"Trained Model saved to: {model_path}")

  id_to_sign = {v: k for k, v in label_map.items()}
  with open(label_map_path, 'wb') as file:
      pickle.dump(id_to_sign, file)
  print(f"Label Map saved to: {label_map_path}")

if __name__ == "__main__":
    train_model()

Loading data from: /content/asl_alphabet_train/asl_alphabet_train
Processing class: asl_alphabet_train (ID: 0)

Feature extraction complete. Total samples: 0
No features extracted. Check dataset path and image content.


In [17]:
!git add train_model.py
!git commit -m "adding full training pipeline"

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


FAST API BACKEND

In [20]:
%%writefile app.py
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
import io
from PIL import Image

MODEL_PATH = 'models/asl_classifier.pkl'
LABEL_MAP_PATH = 'models/label_map.pkl'

# Load model and label map
try:
    with open(MODEL_PATH, 'rb') as f:
        model = pickle.load(f)
    with open(LABEL_MAP_PATH, 'rb') as f:
        id_to_sign = pickle.load(f)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None
    id_to_sign = {0: "ERROR"}

# Initialize MediaPipe Hands for the API
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

#feature extraction for api
def extract_landmarks_from_image(image_bytes):
    img = Image.open(io.BytesIO(image_bytes))
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    image_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        landmark_vector = []
        wrist_x = hand_landmarks.landmark[0].x
        wrist_y = hand_landmarks.landmark[0].y

        for landmark in hand_landmarks.landmark:
            landmark_vector.extend([landmark.x - wrist_x, landmark.y - wrist_y])

        return np.array(landmark_vector)
    return None

#backend function
@app.post("/predict")
async def predict_sign(file: UploadFile = File(...)):
    if model is None:
        return {"error": "Model not loaded"}

    image_bytes = await file.read()
    features = extract_landmarks_from_image(image_bytes)

    if features is None:
        return {"sign": "No Hand Detected", "confidence": 0.0}

    try:
        features = features.reshape(1, -1)

        prediction_id = model.predict(features)[0]
        probabilities = model.predict_proba(features)[0]
        confidence = np.max(probabilities)
        predicted_sign = id_to_sign.get(prediction_id, "Unknown")

        return {
            "sign": predicted_sign,
            "confidence": float(confidence),
        }
    except Exception as e:
        return {"error": f"Prediction failed: {e}"}

@app.get("/")
def read_root():
    return {"message": "ASL Recognition API is running!"}

Writing app.py


In [21]:
!git add app.py
!git commit -m "api: Create FastAPI backend (app.py) for real-time prediction"

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [22]:
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
